[参考](https://blog.csdn.net/han_xiaoyang/article/details/50629608)。

因为所有的变量都是独热编码，所以可以使用伯努利贝叶斯。

In [3]:
import pandas as pd
import numpy as np

In [4]:
%%time
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'])

CPU times: user 3.94 s, sys: 402 ms, total: 4.34 s
Wall time: 4.45 s


In [51]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


## 只选择  3 个特征

### 选择的特征有 DayOfWeek、PdDistrict、hour

In [11]:
# train['hour'] = train.Dates.dt.hour
# pd.get_dummies(train, columns=['DayOfWeek', 'PdDistrict', 'hour'], drop_first=True)
# 因子化星期几、街区、小时等特征
days = pd.get_dummies(train['DayOfWeek'])
district = pd.get_dummies(train['PdDistrict'])
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour)

In [12]:
# 组合特征
X = pd.concat([hour, days, district], axis=1)

In [13]:
y = train['Category']

In [14]:
features = list(X.columns)

In [15]:
len(features)

41

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6)

In [17]:
%%time
from sklearn.naive_bayes import BernoulliNB
# 朴素贝叶斯建模，计算 log_loss
model = BernoulliNB()
model.fit(X_train, y_train)

CPU times: user 2.27 s, sys: 211 ms, total: 2.48 s
Wall time: 2.24 s


In [18]:
y_pred_proba = model.predict_proba(X_test)
y_pred_proba

array([[0.00783589, 0.11355752, 0.00220953, ..., 0.05715589, 0.03772912,
        0.02098816],
       [0.00088779, 0.09087348, 0.00022099, ..., 0.07500372, 0.05515358,
        0.01193892],
       [0.00451893, 0.10145117, 0.00014566, ..., 0.05059517, 0.04206183,
        0.0077054 ],
       ...,
       [0.00146778, 0.07057754, 0.00012945, ..., 0.03861903, 0.06114235,
        0.0085461 ],
       [0.00075216, 0.07849514, 0.00048058, ..., 0.05794842, 0.0628072 ,
        0.00946624],
       [0.00699874, 0.11476767, 0.00038862, ..., 0.05240052, 0.0448625 ,
        0.00816793]])

In [44]:
d = {name: idx for idx, name in enumerate(model.classes_)}
y_test_idx = y_test.map(d)

In [45]:
res = 0
for idx,proba in zip(y_test_idx,y_pred_proba):
    res +=np.log(proba[idx])

In [46]:
-res / len(y_test_idx)

2.583389227940156

In [47]:
from sklearn.metrics import log_loss

log_loss(y_test, y_pred_proba)

2.5833892279403154

## 在测试数据集上运行

In [48]:
# 因子化星期几、街区、小时等特征
days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour)

In [49]:
data_test = pd.concat([days, district, hour], axis=1)
data_test.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [50]:
y_pred_proba = model.predict_proba(data_test)

In [51]:
model.classes_

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype='<U27')

In [52]:
result = pd.DataFrame(y_pred_proba, columns=model.classes_)
submission = pd.concat([test.Id, result], axis=1)
submission.to_csv('../output/BernoulliNB.csv', index=None)